In [1]:
import pandas as pd

# import api keys
from api_keys import flight_api_key, OWM_key

## Flights API

In [2]:
from datetime import datetime, timedelta

airport_icoa = "EDDB"
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=9)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"


import requests

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
response.json()

{'departures': [{'departure': {'scheduledTimeLocal': '2022-10-07 14:10+02:00',
    'scheduledTimeUtc': '2022-10-07 12:10Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'ENGM', 'iata': 'OSL', 'name': 'Oslo'},
    'scheduledTimeLocal': '2022-10-07 15:50+02:00',
    'actualTimeLocal': '2022-10-07 15:56+02:00',
    'scheduledTimeUtc': '2022-10-07 13:50Z',
    'actualTimeUtc': '2022-10-07 13:56Z',
    'quality': ['Basic', 'Live']},
   'number': 'DY 1105',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Boeing 737-800'},
   'airline': {'name': 'Norwegian Air Shuttle'}},
  {'departure': {'scheduledTimeLocal': '2022-10-07 14:10+02:00',
    'scheduledTimeUtc': '2022-10-07 12:10Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'LTAI', 'iata': 'AYT', 'name': 'Antalya'},
    'scheduledTimeLocal': '2022-10-07 18:30+03:00',
    'scheduledTimeUtc': '2022-10-07 15:30Z',
 

In [3]:
### Option 1
arrivals_berlin = response.json()['arrivals']

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':airport_icoa
    }

# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
1,Luxembourg,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
2,Amsterdam,2022-10-07 14:00+02:00,1,CanceledUncertain,Airbus A319,EDDB
3,Munich,2022-10-07 14:05+02:00,1,Unknown,Bombardier CRJ900,EDDB
4,Istanbul,2022-10-07 14:05+02:00,1,Unknown,Airbus A321,EDDB
...,...,...,...,...,...,...
135,London,2022-10-07 22:35+02:00,1,Unknown,Airbus A320,EDDB
136,Munich,2022-10-07 22:35+02:00,1,Unknown,Airbus A321,EDDB
137,Paris,2022-10-07 22:40+02:00,1,Unknown,Airbus A320,EDDB
138,Madrid,2022-10-07 22:40+02:00,1,Unknown,Airbus A321,EDDB


In [4]:
### Option 2
import pandas as pd
arrivals = pd.json_normalize(response.json()['arrivals'])
# next step: select the columns you want to incude on your database
(
arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model'])
    .assign(icao_code = airport_icoa)
)

,departure.airport.name,arrival.scheduledTimeLocal,arrival.terminal,status,aircraft.model,icao_code
0,Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
1,Luxembourg,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
2,Amsterdam,2022-10-07 14:00+02:00,1,CanceledUncertain,Airbus A319,EDDB
3,Munich,2022-10-07 14:05+02:00,1,Unknown,Bombardier CRJ900,EDDB
4,Istanbul,2022-10-07 14:05+02:00,1,Unknown,Airbus A321,EDDB
...,...,...,...,...,...,...
135,London,2022-10-07 22:35+02:00,1,Unknown,Airbus A320,EDDB
136,Munich,2022-10-07 22:35+02:00,1,Unknown,Airbus A321,EDDB
137,Paris,2022-10-07 22:40+02:00,1,Unknown,Airbus A320,EDDB
138,Madrid,2022-10-07 22:40+02:00,1,Unknown,Airbus A321,EDDB


## Wheather API

In [5]:
city = "Berlin"
country = "DE"

### Connecting to OWM API

In [6]:
# achieve the same result with the wather api
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>

In [7]:
forecast_api = response.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-10-07 15:00:00,18.04,3.65,0.0,0,0,"Berlin,DE"
1,2022-10-07 18:00:00,15.33,2.62,0.0,0,0,"Berlin,DE"
2,2022-10-07 21:00:00,12.76,2.58,0.0,0,0,"Berlin,DE"
3,2022-10-08 00:00:00,12.02,2.41,0.0,0,0,"Berlin,DE"
4,2022-10-08 03:00:00,11.08,2.45,0.0,0,0,"Berlin,DE"


## Population data

In [11]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata

# cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Leipzig','Cologne','Dresden','Hannover','Paris', 'Barcelona','Lisbon','Madrid']
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)
# df_cities = df_cities.set_index('city')
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"907,976","1,558,755",NaN,52°22′N,4°54′E
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,719,000[4]","2,871,133[2][3]",38°43′31″N,9°09′00″W
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,275,406",NaN,"2,709,418[4]",50°05′15″N,14°25′17″E
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,951,354","2,890,577",48°12′N,16°22′E
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,211,000[2]","6,791,667[1]",40°25′00″N,03°42′09″W


## Airports data

In [13]:
import pandas as pd

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [14]:
airports_cities.query('municipality == "Berlin"')

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
20244,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE"


## Check the tables

In [15]:
arrivals_berlin.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
1,Luxembourg,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
2,Amsterdam,2022-10-07 14:00+02:00,1,CanceledUncertain,Airbus A319,EDDB
3,Munich,2022-10-07 14:05+02:00,1,Unknown,Bombardier CRJ900,EDDB
4,Istanbul,2022-10-07 14:05+02:00,1,Unknown,Airbus A321,EDDB


In [16]:
arrivals_berlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dep_airport         140 non-null    object
 1   sched_arr_loc_time  140 non-null    object
 2   terminal            140 non-null    object
 3   status              140 non-null    object
 4   aircraft            140 non-null    object
 5   icao_code           140 non-null    object
dtypes: object(6)
memory usage: 6.7+ KB


In [ ]:
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-03-31 12:00:00,7.54,6.81,0.0,0.0,0,"Berlin,DE"
1,2022-03-31 15:00:00,7.54,6.72,0.0,0.0,0,"Berlin,DE"
2,2022-03-31 18:00:00,6.15,6.62,0.0,0.0,0,"Berlin,DE"
3,2022-03-31 21:00:00,3.81,6.85,0.0,0.0,0,"Berlin,DE"
4,2022-04-01 00:00:00,1.48,6.39,0.0,0.0,0,"Berlin,DE"


In [17]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   datetime                  40 non-null     object 
 1   temperature               40 non-null     float64
 2   wind                      40 non-null     float64
 3   prob_perc                 40 non-null     float64
 4   rain_qty                  40 non-null     int64  
 5   snow                      40 non-null     int64  
 6   municipality_iso_country  40 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 2.3+ KB


In [18]:
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [19]:
airports_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 10890 to 68323
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      624 non-null    object 
 1   latitude_deg              624 non-null    float64
 2   longitude_deg             624 non-null    float64
 3   iso_country               623 non-null    object 
 4   iso_region                624 non-null    object 
 5   municipality              619 non-null    object 
 6   icao_code                 619 non-null    object 
 7   iata_code                 617 non-null    object 
 8   municipality_iso_country  618 non-null    object 
dtypes: float64(2), object(7)
memory usage: 48.8+ KB


In [20]:
cities = airports_cities.filter(['municipality','iso_country','municipality_iso_country']).drop_duplicates()
cities.head()

,municipality,iso_country,municipality_iso_country
10890,Honiara,SB,"Honiara,SB"
12461,Port Moresby,PG,"Port Moresby,PG"
12981,Reykjavík,IS,"Reykjavík,IS"
13028,Prishtina,XK,"Prishtina,XK"
17254,"Xintai, Tai'an",CN,"Xintai, Tai'an,CN"


In [21]:
airports_cities.merge(arrivals_berlin, on='icao_code', how='inner').merge(weather_data, on='municipality_iso_country', how='inner').head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country,dep_airport,sched_arr_loc_time,terminal,status,aircraft,datetime,temperature,wind,prob_perc,rain_qty,snow
0,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,2022-10-07 15:00:00,18.04,3.65,0.0,0,0
1,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,2022-10-07 18:00:00,15.33,2.62,0.0,0,0
2,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,2022-10-07 21:00:00,12.76,2.58,0.0,0,0
3,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,2022-10-08 00:00:00,12.02,2.41,0.0,0,0
4,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,2022-10-08 03:00:00,11.08,2.45,0.0,0,0


In [22]:
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"907,976","1,558,755",NaN,52°22′N,4°54′E
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,719,000[4]","2,871,133[2][3]",38°43′31″N,9°09′00″W
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,275,406",NaN,"2,709,418[4]",50°05′15″N,14°25′17″E
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,951,354","2,890,577",48°12′N,16°22′E
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,211,000[2]","6,791,667[1]",40°25′00″N,03°42′09″W


In [23]:
df_cities['municipality_iso_country'] = [
    'Berlin,DE',
    'Paris,FR',
    'Amsterdam,NL',
    'Barcelona,ES',
    'Rome,IT',
    'Lisbon,PT',
    'Prague,CZE',
    'Vienna,AT',
    'Madrid,ES'
]

## Update data into database

In [24]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   datetime                  40 non-null     object 
 1   temperature               40 non-null     float64
 2   wind                      40 non-null     float64
 3   prob_perc                 40 non-null     float64
 4   rain_qty                  40 non-null     int64  
 5   snow                      40 non-null     int64  
 6   municipality_iso_country  40 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 2.3+ KB


### `sqlalchemy`

#### Establish the connection

In [30]:
import sqlalchemy
import pymysql

In [31]:
schema="data_engineering"
host="127.0.0.1"
user="root"
password="root"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Update the tables

In [32]:
(
df_cities
    .dropna()
    .rename(
        columns={
            'lat':'latitude',
            'long':'longitude'
            }
        )
    .to_sql('cities', con=con, if_exists='append', index=False)
    )

2

In [33]:
airports_cities.dropna().to_sql('airports_cities', if_exists='append', con=con, index=False)

611

In [34]:
weather_data.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)

40

In [35]:
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Riga,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
1,Luxembourg,2022-10-07 14:00+02:00,0,Unknown,Boeing 737-800,EDDB
2,Amsterdam,2022-10-07 14:00+02:00,1,CanceledUncertain,Airbus A319,EDDB
3,Munich,2022-10-07 14:05+02:00,1,Unknown,Bombardier CRJ900,EDDB
4,Istanbul,2022-10-07 14:05+02:00,1,Unknown,Airbus A321,EDDB
...,...,...,...,...,...,...
135,London,2022-10-07 22:35+02:00,1,Unknown,Airbus A320,EDDB
136,Munich,2022-10-07 22:35+02:00,1,Unknown,Airbus A321,EDDB
137,Paris,2022-10-07 22:40+02:00,1,Unknown,Airbus A320,EDDB
138,Madrid,2022-10-07 22:40+02:00,1,Unknown,Airbus A321,EDDB


In [36]:
import numpy as np
(
arrivals_berlin
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))

140